In [83]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
import tempfile
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [84]:
# Define stoploss and take profit
stoploss = .05
takeprofit = .12
list_nPCs = [10, 15, 20, 25, 30, 35, 40]

In [85]:
# load the data
trainset_final = pd.read_csv('./Data_JJ/TrainSet_final_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))
trainset = pd.read_csv('./Data_JJ/TrainSet_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))

validation_set_final = pd.read_csv('./Data_JJ/ValidationSet_final_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))
validation_set = pd.read_csv('./Data_JJ/ValidationSet_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))

X_test = validation_set_final
y_test = validation_set["result"]

In [86]:
# Hotfix function
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

def make_keras_picklable():
    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))
    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [87]:
# (b) Build and train several models different amounts of PCs
for nPCs in list_nPCs:
    print(nPCs)
    X = trainset_final.iloc[:, :nPCs]
    y = trainset["result"]

    # Build model and train it
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=nPCs))
    #Second, third and fourth  hidden Layers
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

    #Output Layer
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    #Compiling the neural network
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    #Fitting the data to the training dataset
    classifier.fit(X,y, batch_size=500, epochs=75, verbose =1)

    pk.dump(classifier, open("./Models_JJ/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit),"wb"))

10
Epoch 1/75
1/1 [==============================] - 1s 774ms/step - loss: 0.6931 - accuracy: 0.5158
Epoch 2/75
1/1 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.5113
Epoch 3/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5113
Epoch 4/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5113
Epoch 5/75
1/1 [==============================] - 0s 14ms/step - loss: 0.6930 - accuracy: 0.5113
Epoch 6/75
1/1 [==============================] - 0s 6ms/step - loss: 0.6930 - accuracy: 0.5113
Epoch 7/75
1/1 [==============================] - 0s 6ms/step - loss: 0.6929 - accuracy: 0.5113
Epoch 8/75
1/1 [==============================] - 0s 6ms/step - loss: 0.6929 - accuracy: 0.5113
Epoch 9/75
1/1 [==============================] - 0s 10ms/step - loss: 0.6928 - accuracy: 0.5113
Epoch 10/75
1/1 [==============================] - 0s 11ms/step - loss: 0.6928 - accuracy: 0.5113
Epoch 11/75
1/1 [==============

In [88]:
# (c) Test onto the testset : we compare all models and store results in a csv file
accuracies, nPCs_list = [], []
for nPCs in list_nPCs:
    print(nPCs)
    with open("./Models_JJ/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit), 'rb') as f:
        clf = pk.load(f)
    # Compute predictions on testset
    preds = (clf.predict(validation_set_final.iloc[:, :nPCs]) > 0.5)*1

    # Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
    validation_set1 = validation_set[preds == 1].copy()
    accuracies.append(np.mean(preds == list(validation_set1['result'])))
    nPCs_list.append(nPCs)

10
15
20
25
30
35
40


In [89]:
recap = pd.DataFrame({'nPCs' : list(nPCs_list), 'Accuracy' : (list(accuracies))})
recap.to_csv('./Results_JJ/Comparative_All_models_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit), index = False)
print(recap)

   nPCs  Accuracy
0    10  0.416667
1    15  0.538095
2    20  0.716667
3    25  0.509259
4    30  0.483333
5    35  0.497312
6    40  0.371212
